# PPD: Programação com CUDA

Hélio - DC/UFSCar - 2023

https://developer.nvidia.com/blog/easy-introduction-cuda-c-and-c/

https://developer.nvidia.com/blog/even-easier-introduction-cuda/

In [ ]:
! nvidia-smi -L
# ! echo && nvidia-smi

# Blocos e Threads

A execução de instruções em GPU com CUDA é feita no modelo **SIMT**, criando *threads* a partir de funções definidas como *kernels*.  O **número** e a **organização das *threads*** que vão executar instâncias do *kernel* simultaneamnte são determinados como parâmetros na ativação do *kernel*.

* As *threads* são organizadas em **blocos**. A estrutura dos blocos é definida por uma variável do tipo ***dim3***, que possibilita arranjos em 1, 2 ou 3 dimensões.

* Além disso, é possível definir uma grade (***grid***) de blocos, que podem ser organizados em estruturas lógicas de 1, 2 ou 3 dimensões.

Assim, há 2 parâmetros que são passados na ativação de um *kernel*. O primeiro indica a estruturação do ***grid***, ou seja, quantos blocos de *threads* serão usados e como esses blocos estarão organizados logicamente. O segundo parâmetro indica como as *threads* estarão organizadas dentro de cada bloco.

```c
kernel_function << grid, bloco >>
```
ou seja,
```c
kernel_function << #blocos, #threads_por_bloco >>
```

Durante a execução das *threads*, há funções que permitem a cada uma saber quais são as dimensões do bloco a que essa *thread* pertence e qual é o posicionamento desta *thread* dentro deste bloco, nas dimensões X, Y e Z.

Também é possível saber qual é a organização do *grid* e qual é o posicionamento do bloco desta *thread* em relação ao *grid*.


* ***Thread*** dentro do bloco: **threadIdx.x**, **threadIdx.y**, **threadIdx.z**

* **Bloco** dentro do *grid*: **blockIdx.x**, **blockIdx.y**, **blockIdx.z**

* **Dimensões** do bloco: **blockDim.x**, **blockDim.y**, **blockDim.z**

* **Dimensões** do grid em blocos: **gridDim.x**, **gridDim.y**, **gridDim.z**

<br>

Há 2 fatores associados a essa forma de organização lógica das *threads*. Um aspecto é poder mapear o identificador da *thread* com a organização de estruturas de dados multidimensionais, como vetores, matrizes e volumes.

Outro aspecto da organização das *threads* em blocos é o mapeamento dos blocos aos elementos de processamento das GPUs.

<br>

No exemplo a seguir, vê-se a ativação do *kernel* e a configuração do *grid* e das *threads* dentro dos blocos.

As variáveis **threadIdx** e **blockIdx** indicam a posição lógica de cada *thread* na estrutura.


In [ ]:
%%writefile id.cu

#include <stdio.h>

/*
typedef struct {
   unsigned int x;
   unsigned int y;
   unsigned int z;
} dim3;
*/


__global__
void checkIndex(void)
{
  printf("threadIdx:(%d, %d, %d) blockIdx:(%d, %d, %d) blockDim:(%d, %d, %d) "
      "gridDim:(%d, %d, %d)\n", threadIdx.x, threadIdx.y, threadIdx.z,
      blockIdx.x, blockIdx.y, blockIdx.z, blockDim.x, blockDim.y,
      blockDim.z, gridDim.x, gridDim.y, gridDim.z);
}

int
main(int argc, char **argv)
{
  dim3 grid = {1,1,3};    // organização dos blocos de threads
  dim3 block = {2,2,2};   // organização das threads em cada bloco

  checkIndex <<< grid, block >>>();

  cudaDeviceSynchronize();

  return (0);
}


Writing id.cu


In [ ]:
# ! if [ ! id -nt id.c ]; then nvcc id.cu -o id  -Wno-deprecated-gpu-targets -gencode=arch=compute_37,code=sm_37; fi
! nvcc id.c -o id -O3
! ./id

threadIdx:(0, 0, 0) blockIdx:(0, 0, 0) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(1, 0, 0) blockIdx:(0, 0, 0) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(0, 1, 0) blockIdx:(0, 0, 0) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(1, 1, 0) blockIdx:(0, 0, 0) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(0, 0, 1) blockIdx:(0, 0, 0) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(1, 0, 1) blockIdx:(0, 0, 0) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(0, 1, 1) blockIdx:(0, 0, 0) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(1, 1, 1) blockIdx:(0, 0, 0) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(0, 0, 0) blockIdx:(0, 0, 1) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(1, 0, 0) blockIdx:(0, 0, 1) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(0, 1, 0) blockIdx:(0, 0, 1) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(1, 1, 0) blockIdx:(0, 0, 1) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(0, 0, 1) blockIdx:(0, 0, 1) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(1

# Escolhendo as dimensões do kernel

Devido à forma em que é organizada a arquitetura da GPU, há limitações sobre as dimensões máximas de *grids* e de blocos, como se vê nas informações obtidas com [cudaGetDeviceProperties](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html).

```c
__host__ cudaError_t cudaGetDeviceProperties ( cudaDeviceProp* prop, int  device )

// Returns information about the compute-device.

// prop: Properties for the specified device
// device: Device number to get properties for

// Returns in *prop the properties of device dev. The cudaDeviceProp structure is defined as:

struct cudaDeviceProp {
  char name[256];
  cudaUUID_t uuid;
  ...
  int cudaDeviceProp::maxThreadsPerBlock; // Maximum number of threads per block
  int cudaDeviceProp::maxThreadsDim[3]; // Maximum size of each dimension of a block
  int cudaDeviceProp::maxGridSize[3];  // Maximum size of each dimension of a grid
  ...
  int cudaDeviceProp::maxThreadsPerMultiProcessor; // Maximum resident threads per multiprocessor
  ...
}
```


In [ ]:
%%writefile max-dims.cu

#include <stdio.h>

int main(void)
{
	cudaSetDevice(0);
	cudaDeviceProp prop;
	cudaGetDeviceProperties(&prop,0);

 	printf("Modelo do Device: %s\n",prop.name);
  printf("Número de SMs: %d\n",prop.multiProcessorCount);
  printf("\nmaxGridSize: %d,%d,%d\n",
	 			prop.maxGridSize[0], prop.maxGridSize[1],prop.maxGridSize[2]);
	printf("maxThreadsDim: %d,%d,%d\n",
	 			prop.maxThreadsDim[0], prop.maxThreadsDim[1],prop.maxThreadsDim[2]);
  printf("maxThreadsPerBlock: %d\n",prop.maxThreadsPerBlock);
  printf("maxThreadsPerMultiProcessor: %d\n",prop.maxThreadsPerMultiProcessor);

	return 0;
}

Writing max-dims.cu


In [ ]:
! nvcc max-dims.cu -o max-dims && ./max-dims

Modelo do Device: Tesla T4
Número de SMs: 40

maxGridSize: 2147483647,65535,65535
maxThreadsDim: 1024,1024,64
maxThreadsPerBlock: 1024
maxThreadsPerMultiProcessor: 1024



Como se vê pelos dados obtidos com a chamada *cudaGetDeviceProperties*, as dimensões **máximas de cada bloco** estão limitadas a \< 1024, 1024, 64 \>. Porém, a soma do número de *threads* nessas 3 dimensões não pode ultrapassar **1024**, como retornado em **maxThredsPerBlock**.

Ou seja, pode-se escolher qualquer valor entre 1..1024 para a primeira dimensão do bloco, qualquer valor entre 1..1024 para a segunda dimensão, e qualqer valor entre 1..64 para a 3a dimensão, desde que **d1\*d2\*d3 <= 1024**.

Já o número máximo de blocos é MUITO grande, com dimensões máximas \< 2147483647, 65535, 65535 \>.

Deste modo, o número total de *threads* que uma aplicação pode ter é 2147483647 \* 65535 \* 65535 \* 1024.

São muitas *threads*, cabendo ao programador escolher a melhor organização lógica para isso.


# Escolhendo a organização das *threads*

Como visto, é possível utilizar difentes formas de organizar um número fixo de *threads* em grades com diferentes números de blocos, sendo que cada bloco pode ter diferentes números de *threads*, organizadas em 1, 2 ou 3 dimensões.

O ponto aqui é considerarmos que podemos definir uma *thread* para o cálculo de cada elemento de uma estrutura vetorial a ser manipulada. Independentemente do número de *cores* (núcleos de processamento da GPU), podemos pensar como se todas as *threads* fossem executadas ao mesmo tempo.  

Sabendo que a organização das *threads* ocorre na forma de uma **grade de blocos**, a conta que precisa ser feita é:

* definir o número de *threads* do bloco
* dividir o número de elementos a manipular pelo tamanho do bloco, resultando no tamanho da grade

É claro, né?! Número de threads = núm_blocos * tam_bloco :-)

<br>

Para que todos os elementos sejam calculados, ou seja, para que haja ao menos uma *thread* por elemento a calcular, é preciso **arredondar para cima** o **resultado desta divisão**.

Caso as *threads* do bloco sejam organizadas em apenas 1 dimensão (x), isso pode ser feito de duas formas:

* somando o tamanho do boco menos um elemento (block.x -1) ao número de elementos, se quisermos fazer esta conta usando valores do tipo inteiro (int), que descarta o resto;

* fazendo a divisão de valores em ponto flutuante, e considerando o próximo valor inteiro, se o resultado for fracionado, usando a função ceil().


```c
dim3 block, grid;

block.x = 1024;
grid.x = (nElem + block.x -1) / block.x;    // divisão com valores inteiros
// ou
grid.x = ceil ( (float)nElem / (float)block.x );  // cálculo com float, pegando inteiro seguinte quando resultado é fracionado
```

Em geral, a primeira opção é usada, dado que o cálculo de valores inteiros é menos custoso do que operações em ponto flutuante.


# Organização de *threads* numa soma de matrizes

Consideremos uma soma de 2 matrizes com 1000 x 1000 elementos.

Pensando na decomposição máxima que esse problema pode sofrer, há 1.000.000 somas a serem feitas, todas independentes e passíveis de serem realizadas simultaneamente.

Como vimos, contudo, não dá para criar todas essas *threads* usando apenas as **dimensões do bloco** de *threads*, que está limitado a 1024.

Pensando apenas em resolver a conta, uma possível estratégia seria então considerar um bloco com 1024 *threads* e criar um número de blocos necessários para acomodar todas as *threads*. Assim, a composição do *grid* poderia ser algo como {1000000/1024, 1, 1} .

Vale observar que, como estamos tratando de divisão inteira, pode ser preciso usar o **teto** (*ceil*()) desta divisão. Para tanto, é comum fazer uma conta como (1000000+1023) / 1024. Neste caso, a parte inteira do resultado é 977.

<br>

Assim, uma possível ativação do *kernel* poderia dimensionar as *threads* da seguinte maneira:

```c
  int threadsPerBlock = 1024;
  int blocksPerGrid = ( 1000000 + threadsPerBlock - 1) / threadsPerBlock;
  // ou int blocksPerGrid = (int) ceil (1000000. / (double)threadsPerBlock );

  MatAdd <<< blocksPerGrid, threadsPerBlock >>> ();
```

<br>

Ah, neste caso, temos uma *thread* para cada soma, mas falta definir qual *thread* vai manipular qual elemento da matriz (bi-dimensional)...

<br>

Vale lembrar que tanto a organização das *threads* de um bloco quanto a composição dos blocos do *grid* são estruturas que podem ter 1, 2 ou 3 dimensões. Assim, será que há alguma outra forma mais vantajosa para organizarmos as *threads*?

Para responder essa pergunta, é preciso conhecer alguns aspectos adicionais sore a arquitetura CUDA e a organização física dos processadores.

# Cuda SMs

Ao longo do tempo, diferentes modelos de GPU foram desenvolvidos, com capacidades variadas. Entre os aspectos distintos nas diferentes versões está o número de multiprocessadores (**SMs: *Stream Multiprocessors***).

Esse valor e o número de elementos de processamento dentro de cada SM mudam de acordo com a [*capability*](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities) da placa. Essas informações podem ser obtidas com a chamada [cudaGetDeviceProperties](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1g1bf9d625a931d657e08db2b4391170f0).

In [ ]:
%%writefile dev-cap.cu

#include <stdio.h>

int main(void)
{
	cudaSetDevice(0);
	cudaDeviceProp prop;
	cudaGetDeviceProperties(&prop,0);

 	printf("Modelo do Device: %s\n\n",prop.name);
  printf("Computing capability: %d.%d\n",prop.major, prop.minor);
  printf("Número de SMs (multiProcessorCounr): %d\n",prop.multiProcessorCount);
  printf("warpSize: %d\n\n", prop.warpSize);

	return 0;
}

Writing dev-cap.cu


In [ ]:
! nvcc dev-cap.cu -o dev-cap && ./dev-cap

Modelo do Device: Tesla T4

Computing capability: 7.5
Número de SMs (multiProcessorCounr): 40
warpSize: 32



# Execução de *threads*

https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#hardware-implementation

Quando um programa CUDA executado no *host* (CPU) ativa a execução de um *kernel*, é especificada uma grade (*grid*) de blocos, cada um contendo uma organização de *threads*. Cabe ao programador decidir como os dados a manipular serão divididos entre essas *threads*, ou seja, quantas *threads* são necessárias para manipular os dados.

Para isso, o programador pode considerar a GPU de forma abstrata, como se ela tivesse tantos núcleos (*cores*) que fosse possível executar todas essas *threads* ao mesmo tempo, cada uma manipulando um dado em paralelo, numa única etapa.

Como vimos, contudo, o número de processadores (*CUDA cores*) em uma GPU pode [variar](https://www.nvidia.com/pt-br/geforce/graphics-cards/compare/), estando comumente limitados a alguns poucos milhares [1].

Além disso, a organização desses processadores numa GPU também pode variar. A arquitetura das GPUs Nvidia é construída na forma de um conjunto escalável de blocos multiprocessadores, chamados de ***Streaming Multiprocessors***, ou **SMs**. Cada um desses SMs tem tipicamente 32 processadores (*CUDA cores*).

<br>

Contudo, como é que as *threads* especificadas na ativação de um *kernel* são atribuídas aos processadores da GPU para execução?

<br>

As atribuições para execução ocorrem em blocos de *threads*. Os blocos de *threads* do *grid* especificado são atribuídos aos multiprocessadores (SMs), aparentemente em rodadas (*round-robin*), e não migram. Assim, múltiplos blocos de *threads* podem ser executados de forma paralela nos vários SMs, sendo que as *threads* de um bloco executam de maneira paralela dentro de um SM.

É claro, contudo, que o número de blocos pode ser superior ao número de SMs e o número de *threads* pode ser superior ao número de *CUDA cores* em um SM. Assim, pode haver uma **concorrência** tanto na execução das *threads* de um bloco quanto na execução dos blocos.

À medida que a execução dos blocos de *thread* é concluída, novos blocos são ativados (*launched*) nos multiprocessadores que ficaram vazios.

Esse gerenciamento é feito internamente pela GPU, sem intereferência do programador.

<br>

Cada multiprocessador é projetado para executar centenas de *threads* concorrentemente, no modelo SIMT (*Single-Instruction, Multiple-Thread*).

As *threads* em um multiprocessador (SM) são gerenciadas e **executadas de forma paralela em grupos de 32**, chamados ***warps***. As *threads* de um *warp* são colocadas em execução a partir do mesmo endereço do programa, mas cada *thread* tem seu ponteiro de instruções e estado dos registradores próprios, de forma que podem sofrer desvios e executar de forma independente.

Quando um multiprocessador recebe um bloco de *threads* para executar, ele o particiona em *warps*, que serão escalonadas para execução por um escalonador de *warps*. O particionamento das *threads* em *warps* ocorre sempre da mesma maneira, atribuindo a cada *warp* as *threads* com identificadores consecutivos.

Uma *warp* executa uma instrução comum de cada vez, de forma que a eficiência máxima é obtida quando todas as 32 *threads* de uma *warp* seguem no mesmo fluxo de execução. Se as *threads* numa *warp* divergem em desvios condicionais, a *warp* executa cada caminho seguido, desabilitando as *threads* cujo fluxo não está naquele caminho. *Warps* distintas, contudo, têm fluxos de execução próprios, de maneira disjunta das demais *warps*.

Embora nas primeiras versões de arquiteturas de GPUs um único ponteiro de instruções era mantido para todas as *threads* de uma *warp*, em versões mais recentes, os fluxos podem ser distintos nas *threads*. Assim, do ponto de vista da execução do código, o comportamento das *threads* vai ser correto mesmo que o programador ignore o modelo de execução SIMT. De todo modo, melhores desempenhos podem ser obtidos se os fluxos de execução nas *threads* não divergirem.

### **Em suma**:

* Todos os SMs de uma GPU trabalham em paralelo, de forma independente

* O particionamento do *grid* de *threads* de um *kernel* é feito atribuindo blocos inteiros aos SMs.

* Dentro de um SM, as *threads* de cada bloco são divididas em grupos de 32 *threads* para execução, chamados *warps*.

* A execução de um *warp* ocorre no modelo SIMT, com todas as *threads* executando a mesma sequência de instruções.
  * Desvios condicionais no fluxo de instruções das *threads* de uma *warp* podem fazer com que algumas *threads*, que não seguiram o mesmo caminho no código, fiquem temporariamente paradas.

* *Threads* em um SM compartilham a memória deste SM, que pode ser usada para comunicação eficiente entre elas.

* Cooperações e sincronizações só são possíveis entre *threads* do mesmo bloco.

<!--
### Algumas conclusões    (minhas, requerem mais investigação... *hélio*)

* O paralelismo efetivo máximo corresponde a 32 threads de um *warp* vezes o número de SMs da GPU
* Dentro de uma SM, até 32 *threads* de uma *warp* vão estar em execução de uma vez (pois esse é o número de *processing elements*). Na verdade, depende do número de CUDA cores em cada SM... Esse valor pode ser 64....
*
-->

<br>

[1] [Nvidia GPUs sorted by CUDA cores](https://gist.github.com/cavinsmith/ed92fee35d44ef91e09eaa8775e3284e)<br>
[2] https://forums.developer.nvidia.com/t/what-is-cores-per-sm/29997

# Hierarquia de *threads*

https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#thread-hierarchy

***Thread Hierarchy***

*For convenience, threadIdx is a 3-component vector, so that threads can be identified using a one-dimensional, two-dimensional, or three-dimensional thread index, forming a one-dimensional, two-dimensional, or three-dimensional block of threads, called a thread block. This provides a natural way to invoke computation across the elements in a domain such as a vector, matrix, or volume.*

*The index of a thread and its thread ID relate to each other in a straightforward way:*

* For a one-dimensional block, they are the same;
* for a two-dimensional block of size (Dx, Dy), the thread ID of a thread of index (x, y) is (x + y Dx);
* for a three-dimensional block of size (Dx, Dy, Dz), the thread ID of a thread of index (x, y, z) is (x + y Dx + z Dx Dy).


# Linearização dos índices

É claro que GPUs e seus processadores que operam no modo SIMT têm tudo a ver com o processamento simultâneo de dados de estruturas como matrizes e vetores. Basta determinar uma forma para que cada *thread* na GPU saiba qual dado irá manipular.

O caminho natural para este mapeamento é usar os **índices das *threads* e dos blocos** a que pertencem nessa identificação.

<br>

O programa a seguir visa testar o mapeamento de índices de um vetor manipulado por diferentes blocos de *threads*.

Do ponto de vista da alocação de memória para estruturas bi ou tridimensionais, uma estratégia é alocar os elementos em sequência, como se fosse uma estrutura unidimensional.

Para mapear as posições corretas da estrutura, contudo, é preciso  **linearizar** os índices.

Isso é o que é feito quando uma matriz é tratada como uma sequência de linhas, cada uma composta da dimensão completa das colunas:

M \[ i, j \] = M \[ i \* ncol +j \]

O exemplo a seguir procura ilustrar a linearização dos índices das *threads*, considerando a organização do grid como uma estrutura de blocos, cada um composto de uma estrutura de *threads*.


In [ ]:
%%writefile ind.cu

#include <stdio.h>

/* https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#thread-hierarchy
 *
 * The index of a thread and its thread ID relate to each other in a straightforward way:
 *
 * For a one-dimensional block, they are the same;
 * for a two-dimensional block of size (Dx, Dy), the thread ID of a thread of index (x, y) is (x + y Dx);
 * for a three-dimensional block of size (Dx, Dy, Dz), the thread ID of a thread of index (x, y, z) is (x + y Dx + z Dx Dy).
 */


__global__
void mark(int *vet, int dim)
{
  //  for a three-dimensional block of size (Dx, Dy, Dz), the thread ID of a thread of index (x, y, z) is (x + y Dx + z Dx Dy).
  // int block = blockIdx.x + blockIdx.y * gridDim.x + blockIdx.z * gridDim.x * gridDim.y;
  int block = blockIdx.x * gridDim.y * gridDim.z + blockIdx.y * gridDim.z + blockIdx.z;

  // int thread = threadIdx.x + threadIdx.y * blockDim.x + threadIdx.z * blockDim.x * blockDim.y;
  int thread = threadIdx.x * blockDim.y * blockDim.z + threadIdx.y * blockDim.z + threadIdx.z;

  int ind = block * blockDim.x * blockDim.y * blockDim.z + thread;

  printf("%d,%d,%d / %d,%d,%d : %d\n", blockIdx.x, blockIdx.y,blockIdx.z,
        threadIdx.x, threadIdx.y, threadIdx.z, ind);

  // Testa se índice resultante está dentro do limite do vetor!
  if (ind < dim)
    vet[ind] = ind;
}

int
main(int argc, char **argv)
{
  int i, *vet, *d_vet;
  dim3 grid = {2,2,2};    // organização dos blocos de threads
  dim3 block = {2,2,2};   // organização das threads em cada bloco

  int dim = grid.x * grid.y * grid.z * block.x * block.y * block.z;

  vet = (int *)malloc(dim * sizeof(int));

  // alocação de memória na GPU
  cudaMalloc(&d_vet, dim * sizeof(int));

  // Neste caso, não precisa copiar para a memória da GPU; inicialização na GPU
  // cudaMemcpy(d_vet, vet, dim * sizeof(int), cudaMemcpyHostToDevice);

  mark <<< grid, block >>>(d_vet, dim);

  // Esta é a forma de testar se houve erro na ativação do kernel
  cudaError_t err = cudaGetLastError();
  if (err != cudaSuccess) {
    printf("CUDA Error: %s\n",cudaGetErrorString(err));
    cudaDeviceReset();
    exit(0);
  }

  // Cópia dos dados da memória da GPU para a memória RAM
  cudaMemcpy(vet, d_vet, dim * sizeof(int), cudaMemcpyDeviceToHost);

  // Verifica se índices foram preenchidos corretamente
  for (i=0; i < dim; i++)
    if(vet[i]!=i)
      printf("Erro em %d\n",i);

  cudaFree(d_vet);

  cudaDeviceSynchronize();

  return (0);
}

Writing ind.cu


In [ ]:
# Se ordenarmos o resultado, fica fácil conferir para quem sabe contar em binário :-)
# ! nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 ind.cu -o ind && ./ind | sort
! nvcc ind.cu -o ind && ./ind | sort

0,0,0 / 0,0,0 : 0
0,0,0 / 0,0,1 : 1
0,0,0 / 0,1,0 : 2
0,0,0 / 0,1,1 : 3
0,0,0 / 1,0,0 : 4
0,0,0 / 1,0,1 : 5
0,0,0 / 1,1,0 : 6
0,0,0 / 1,1,1 : 7
0,0,1 / 0,0,0 : 8
0,0,1 / 0,0,1 : 9
0,0,1 / 0,1,0 : 10
0,0,1 / 0,1,1 : 11
0,0,1 / 1,0,0 : 12
0,0,1 / 1,0,1 : 13
0,0,1 / 1,1,0 : 14
0,0,1 / 1,1,1 : 15
0,1,0 / 0,0,0 : 16
0,1,0 / 0,0,1 : 17
0,1,0 / 0,1,0 : 18
0,1,0 / 0,1,1 : 19
0,1,0 / 1,0,0 : 20
0,1,0 / 1,0,1 : 21
0,1,0 / 1,1,0 : 22
0,1,0 / 1,1,1 : 23
0,1,1 / 0,0,0 : 24
0,1,1 / 0,0,1 : 25
0,1,1 / 0,1,0 : 26
0,1,1 / 0,1,1 : 27
0,1,1 / 1,0,0 : 28
0,1,1 / 1,0,1 : 29
0,1,1 / 1,1,0 : 30
0,1,1 / 1,1,1 : 31
1,0,0 / 0,0,0 : 32
1,0,0 / 0,0,1 : 33
1,0,0 / 0,1,0 : 34
1,0,0 / 0,1,1 : 35
1,0,0 / 1,0,0 : 36
1,0,0 / 1,0,1 : 37
1,0,0 / 1,1,0 : 38
1,0,0 / 1,1,1 : 39
1,0,1 / 0,0,0 : 40
1,0,1 / 0,0,1 : 41
1,0,1 / 0,1,0 : 42
1,0,1 / 0,1,1 : 43
1,0,1 / 1,0,0 : 44
1,0,1 / 1,0,1 : 45
1,0,1 / 1,1,0 : 46
1,0,1 / 1,1,1 : 47
1,1,0 / 0,0,0 : 48
1,1,0 / 0,0,1 : 49
1,1,0 / 0,1,0 : 50
1,1,0 / 0,1,1 : 51
1,1,0 / 1,0,0 : 52
1,1